<a href="https://colab.research.google.com/github/sanvidhaharibhakta/Practicals/blob/main/Bubble_sort(static_input).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-pv6pl6lc
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-pv6pl6lc
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=1676fe5a27f88cfdcf54ec9eaea0a882ec79d52744665f50e5ffd80e3fc568fe
  Stored in directory: /tmp/pip-ephem-wheel-cache-0pam9vy4/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [6]:
%%writefile bubblesort.cu

#include <iostream>
#include <vector>
#include <chrono>
#include <cuda_runtime.h>

// CUDA kernel for Bubble Sort
__global__ void bubble_sort(int* d_arr, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x; // Thread index
    for (int i = 0; i < size - 1; i++) {
        int j = idx + i; // Offset to perform the bubble sort step
        if (j < size - 1 && d_arr[j] > d_arr[j + 1]) { // Swap if out of order
            int temp = d_arr[j];
            d_arr[j] = d_arr[j + 1];
            d_arr[j + 1] = temp;
        }
        __syncthreads(); // Synchronize threads within block
    }
}

// Function for Bubble Sort on CPU
void bubble_sort_cpu(int* arr, int size) {
    for (int i = 0; i < size - 1; i++) {
        for (int j = 0; j < size - 1 - i; j++) {
            if (arr[j] > arr[j + 1]) { // Swap if out of order
                int temp = arr[j];
                arr[j] = arr[j + 1];
                arr[j + 1] = temp;
            }
        }
    }
}

int main() {
    // Test data
    int h_arr[] = {64, 34, 25, 12, 22, 11, 90};
    int size = sizeof(h_arr) / sizeof(h_arr[0]);

    // Bubble Sort on CPU
    auto start = std::chrono::high_resolution_clock::now();
    bubble_sort_cpu(h_arr, size);
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration = end - start;
    std::cout << "Sequential Bubble Sort took " << duration.count() << " seconds\n";

    // Copying data to the device for parallel Bubble Sort
    int* d_arr;
    cudaMalloc(&d_arr, size * sizeof(int));
    cudaMemcpy(d_arr, h_arr, size * sizeof(int), cudaMemcpyHostToDevice);

    // Bubble Sort on GPU
    start = std::chrono::high_resolution_clock::now();
    int blockSize = 256; // Threads per block
    int gridSize = (size + blockSize - 1) / blockSize; // Blocks
    bubble_sort<<<gridSize, blockSize>>>(d_arr, size);
    cudaMemcpy(h_arr, d_arr, size * sizeof(int), cudaMemcpyDeviceToHost); // Copy back to host
    end = std::chrono::high_resolution_clock::now();
    duration = end - start;
    std::cout << "Parallel Bubble Sort took " << duration.count() << " seconds\n";

    // Display sorted array
    std::cout << "Sorted Array: ";
    for (int i = 0; i < size; i++) {
        std::cout << h_arr[i] << " ";
    }
    std::cout << std::endl;

    // Free device memory
    cudaFree(d_arr);

    return 0;
}

Writing bubblesort.cu


In [8]:
!nvcc bubblesort.cu -o bubblesort

In [9]:
!./bubblesort

Sequential Bubble Sort took 3.9e-07 seconds
Parallel Bubble Sort took 0.111814 seconds
Sorted Array: 11 12 22 25 34 64 90 
